<a href="https://colab.research.google.com/github/juanflorezVe/COMP9058_lab1/blob/master/r00184264_juanflorez_assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copying vervatim the code from the assignment
import tensorflow as tf
print(tf.__version__)


2.4.1


In [2]:
import tensorflow as tf
import numpy as np

from keras.utils import np_utils

fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)

4423680/4422102 [==============================] - 0s 0us/step
Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)


In [19]:
# from lecture (VectorizedLogistiRegression, page 50) March 9th.... 
# We have a set of trainning of 60000 rows (instances)
# and 748 columns ( Features)
# Our X vector has 784 rows (as fetarues) and 60000 columns (instances)
X = tf.Variable(tr_x.T, dtype = 'float32')
print("X shape is {}".format( X.shape))
# Let's create a vector W1 with the number of weights (784) for each neuron in one row
# so, it means 200 rows and 784 columns ( Features) for layer 1
W1 = tf.Variable(tf.random.normal([ 200,784], mean=0.0, stddev=0.05))
#The bias for the first layer of 200 neurons will be a vecoter of 200 rows (1 per neuron)
# and 1 column
B1 = tf.Variable([0.0]*200)
B1 = tf.transpose(B1)
print ("W1 shape is{}".format(W1.shape))
print ("B1 shape is{}".format(B1.shape))

X shape is (784, 60000)
W1 shape is(200, 784)
B1 shape is(200,)


Question 1, section a. Question1_1_1


In [4]:
# Let's implement the final softmax layer L2

In [5]:
L2 = tf.Variable(tf.random.normal([10, 1], mean=0.0, stddev=0.05))

In [6]:
L2

<tf.Variable 'Variable:0' shape=(10, 1) dtype=float32, numpy=
array([[ 0.00175995],
       [ 0.05755138],
       [ 0.00603519],
       [-0.08266527],
       [ 0.00290031],
       [ 0.02845798],
       [-0.04208099],
       [-0.06879644],
       [-0.0847582 ],
       [-0.0654161 ]], dtype=float32)>

In [7]:
#Let's insert 784 parameters to each neuron
# A = W.T * X + b
b = tf.Variable([0.])
A = tf.matmul(tf.transpose(W), tr_x) + b


In [8]:
A.shape

TensorShape([1, 784])

In [9]:
L2.shape


TensorShape([10, 1])

In [10]:
#Review slide 33 of VectorizedLogisticRgression
